# Azure ML Example

This notebook demonstrates how to train and deploy a BERT-based retrieval-enhanced model using Azure Machine Learning.

In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig

# Load the workspace
ws = Workspace.from_config()

# Create an experiment
experiment = Experiment(workspace=ws, name='bert-retrieval-experiment')

# Create an environment
env = Environment.from_conda_specification(name='bert-retrieval-env', file_path='../environment.yml')

# Create a script config
src = ScriptRunConfig(source_directory='../scripts', script='train.py', environment=env)

# Submit the experiment
run = experiment.submit(config=src)
run.wait_for_completion(show_output=True)

## Deploy the Model

Once the model is trained, you can deploy it using Azure ML.

In [2]:
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig

# Register the model
model = run.register_model(model_name='bert-retrieval-model', model_path='outputs/model')

# Create an inference config
inference_config = InferenceConfig(entry_script='../scripts/score.py', environment=env)

# Define the deployment configuration
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model
service = Model.deploy(workspace=ws, name='bert-retrieval-service', models=[model], inference_config=inference_config, deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

print(service.scoring_uri)